In [11]:
import math
import numpy as np
import seaborn as sns
from time import time
import pandas as pd
import pymcmcstat
import matplotlib.pyplot as plt
from pymcmcstat.MCMC import MCMC
from pymcmcstat.MCMC import DataStructure#,  ModelParameters

"""
m is a resultant brightness
m$_{\odot}$ is a Sun’s magnitude
$\varphi$ is the satellite-sun-sensor solar phase angle (0º < STO < 90º)
A is the cross-sectonal area
$\textcolor{blue}{\rho}$ is the bond albedo
R$\textcolor{orange}{_{m}}$= is the range to target (400km < R$\textcolor{orange}{_m}$ < 2500 km) 
"""

mag_v_sun = -26.72              #Sun's V magnitude of Gaia   Jordi et al. 2010

### Model of brightness

In [3]:
def sum_squares(theta, data):

    value_model = model_magnitud(theta, data)
    residual = data[2] - value_model
    # print(residual)
    return (residual ** 2).sum(axis=0)

def model_magnitud(theta, data_in):
	mag_v_sun = -26.72

	phase_angle = data_in[0].T
	range_sat = data_in[1].T
	cross_section = theta[0]
	albedo = theta[1] 
	mix_coe = theta[2]

   # Change degrees to radian
	degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
	coe_1 = 2/(3*np.pi)
	coe_2 = 1/(4*np.pi)

	phase_angle_rad = degtorad(phase_angle)

	part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
	
	mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coe * part1) + coe_2) + 5*np.log10(range_sat)
	
	# data_out = {'mag' : mag,
	# 		'phase_angle' : phase_angle,
	# 		'cross_sectio' : cross_section,
	# 		'albedo' : albedo,
	# 		'range_sat' : range_sat,
	# 		'mix_coe' : mix_coe
	# }
	
	# all_mag = pd.DataFrame(data_out)
	# print(data_out)
	return mag

### Define model and sum-of-squares function 

theta is a result of mcstat = MCMC()

In [4]:
def sum_squares(theta, data):

    value_model = model_magnitud(theta, data)
    residual = data[2] - value_model
    # print(residual)
    return (residual ** 2).sum(axis=0)
    

In [5]:
path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
data_sat = pd.read_csv(path)
# data_sat.columns

In [6]:
# Constants
# Sun's magnitud
mag_sun = -26.72

# Define data
range_sat = data_sat['range_sat'].to_numpy().T
mag_observation = data_sat['mag_observation'].to_numpy().T
mag_estimate = data_sat['mag_estimate'].to_numpy().T
sat_phase_angle = data_sat['obs_phase_angle'].to_numpy().T

print()
data_x = np.array([sat_phase_angle, range_sat, mag_observation])


# Give format of data
data_in = DataStructure()
data_in.add_data_set(x= data_x, y = data_x[2])


# initialize parameter array
cross_section_int = [0.5, 1.3]
albedo_int = [0., 1.]
mix_coeff_int = [.5, 1.]



In [16]:
data_x[0].T

data_sat['mag_estimate']




0      13.186802
1       9.196008
2       8.586037
3       8.537361
4       9.343130
         ...    
139     4.463076
140     4.668030
141     4.363651
142     4.459842
143     4.883775
Name: mag_estimate, Length: 144, dtype: float64

In [ ]:
import seaborn as sns

sns.jointplot(data=data_sat, x="obs_phase_angle", y="mag_observation")

In [7]:
# Initialize MCMC object
mcstat = MCMC()

# initialize data structure 
# mcstat.data.add_data_set(
#     x=data_x, y=data_y, user_defined_object=model_magnitud
# )
mcstat.data = data_in


method

In [10]:
theta0 = [0.5, 0., 0.5]

mcstat.parameters.add_model_parameter(name='cross_section',
                                      theta0=theta0[0])

mcstat.parameters.add_model_parameter(name='albedo',
                                      theta0=theta0[1])

mcstat.parameters.add_model_parameter(name='mix_coeff',
                                      theta0=theta0[2])

mcstat.parameters.display_model_parameter_settings()

AttributeError: 'ModelParameters' object has no attribute 'display_model_parameter_settings'

In [ ]:
# Define method for estimating the error variance and computing the covariance matrix
mcstat.model_settings.define_model_settings(sos_function = sum_squares)

# Generate samples
mcstat.simulation_options.define_simulation_options(nsimu = 5000,
                                                    updatesigma=True)




In [ ]:
mcstat.run_simulation(use_previous_results=False)

In [ ]:
from pymcmcstat import mcmcplot as mcp
# plot chain panel
mcp.plot_chain_panel(chain, names)
# pairwise correlation
settings = dict(fig=dict(figsize=(4, 4)))
f = mcp.plot_pairwise_correlation_panel(chain, names, settings=settings)